In [9]:
import numpy as np
# from statsmodels.tsa.arima.model import ARIMA

# ----参数设定 Configuration----
# Half-hourly data for 100 days => 100 * 48 = 4800 points
HOURS_PER_DAY   = 24
POINTS_PER_HOUR = 2  # half-hourly
T_DAYS          = 100
T = T_DAYS * HOURS_PER_DAY * POINTS_PER_HOUR

# Number of latent classes and series per class
K                   = 3
SERIES_PER_CLASS    = 50
N = K * SERIES_PER_CLASS  # total customers


# 固定随机种子 fix seed 
rng = np.random.default_rng(42)

t = np.arange(1, T + 1, dtype=float)  # time index starting at 1..T

data_Folder = "Simulated_data\\"


In [10]:
# ---- Components ----

# (1) Seasonality component: daily cycle only, c_t = |sin(pi/24 * t)|  [Eq. (18)]
c_t = np.abs(np.sin(np.pi / 24 * (t)))  # convert half-hour steps to hours

# (2) Base trend component: d_t = phi2 * t^theta + phi1 * t + phi0  [Eq. (19)]
d_t_class_1 = 7e-3 * t + 8
d_t_class_2 = 0.35 * np.sqrt(t) + 8
d_t_class_3 = 7e-7 * (t**2) - 2e-4 * t + 20


# (3) White noise ω_t ~ ARIMA(0,0,0) i.e., i.i.d. white noise; then scaled by beta3 ∈ (9,10)
# ARIMA(0,0,0) is white noise which is equal to noraml distribution
def sample_noise(T):
    beta3 = rng.uniform(9.0, 10.0)  # per paper: randomly generate in (9, 10)
    return beta3 * rng.normal(0, 1, size=T)
    # return 0 

# (4) Temperature-related component T_t:
T_t = np.load(data_Folder+'beijing_temp_halfhour_'+str(T_DAYS-1)+'.npy')

# aaa = sample_noise(T)
# print(aaa.shape)

In [11]:
beta1 = 1
beta2 = 1
beta4 = 1
for index in range(1,11):
    series = []
    for customer in range(SERIES_PER_CLASS):
        customer_1  = beta1*c_t + beta2*d_t_class_1 + sample_noise(T) + beta4*T_t
        customer_2  = beta1*c_t + beta2*d_t_class_2 + sample_noise(T) + beta4*T_t
        customer_3  = beta1*c_t + beta2*d_t_class_3 + sample_noise(T) + beta4*T_t
        series.append(customer_1)
        series.append(customer_2)
        series.append(customer_3)
    series = np.array(series)
    print(series.shape)
    np.save(data_Folder+"simulated_load_"+str(index),series)
a = sample_noise(T)

(150, 4800)
(150, 4800)
(150, 4800)
(150, 4800)
(150, 4800)
(150, 4800)
(150, 4800)
(150, 4800)
(150, 4800)
(150, 4800)


In [12]:
print(d_t_class_1[-1])
print(d_t_class_2[-1])
print(d_t_class_3[-1])

41.6
32.24871130596428
35.168
